In [70]:
from data import wiki
from data import corpus
import alignment

import nltk
import pandas as pd
import pickle
from tqdm import tqdm_notebook as tqdm

tqdm().pandas()

posts = wiki.load_posts()
pairs = corpus.get_reply_pairs(posts)
users = wiki.load_users(posts=posts)

In [90]:
markers = alignment.load_markers()

In [107]:
def scp_alignment_towards_group(markers, group, users, posts):
    markers_ab = list(map(lambda x: x+'_a', markers)) + list(map(lambda x: x+'_b', markers))
    df = pairs[['user_a', 'user_b'] + markers_ab]\
        .merge(users[[group]].add_suffix('_a'), how='left', left_on='user_a', right_index=True)\
        .merge(users[[group]].add_suffix('_b'), how='left', left_on='user_b', right_index=True)
    df.head()
    for m in markers:
        df[m+'_a'] = df[m+'_a'] > 0
        df[m+'_b'] = df[m+'_b'] > 0
        posts[m+'_bool'] = posts[m] > 0
        prior_m = posts[['user',m+'_bool']].groupby('user').mean()
        scp = df[df[m+'_a']][[group+'_a', 'user_b', m+'_b']].groupby([group+'_a', 'user_b']).mean().reset_index(group+'_a').join(prior_m)
        scp['scp'] = scp[m+'_bool'] - scp[m+'_b']
        print(m)
        print(scp[[group+'_a', 'scp']].groupby(group+'_a').mean())
        print()

In [108]:
scp_alignment_towards_group(markers, 'admin', users, posts)

conjunctions
              scp
admin_a          
False    0.005012
True     0.012089

articles
              scp
admin_a          
False    0.001644
True     0.018751

prepositions
              scp
admin_a          
False    0.004992
True     0.018455

adverbs
              scp
admin_a          
False   -0.002145
True     0.012612

quantifiers
              scp
admin_a          
False   -0.033471
True    -0.025629

impersonal_pronouns
              scp
admin_a          
False   -0.014031
True    -0.000033

personal_pronouns
              scp
admin_a          
False    0.018568
True     0.035509

auxiliary_verbs
              scp
admin_a          
False   -0.008239
True     0.013442



In [124]:
threshold = users.centrality.mean() + users.centrality.std()
users['highly_central'] = (users['centrality'] > threshold)
users.highly_central.value_counts()

False    29007
True      1893
Name: highly_central, dtype: int64

In [125]:
scp_alignment_towards_group(markers, 'highly_central', users, posts)

conjunctions
                       scp
highly_central_a          
False             0.006840
True              0.009705

articles
                       scp
highly_central_a          
False             0.003028
True              0.013799

prepositions
                       scp
highly_central_a          
False             0.005411
True              0.016019

adverbs
                       scp
highly_central_a          
False            -0.000134
True              0.010870

quantifiers
                       scp
highly_central_a          
False            -0.035247
True             -0.024755

impersonal_pronouns
                       scp
highly_central_a          
False            -0.012832
True             -0.006710

personal_pronouns
                       scp
highly_central_a          
False             0.020581
True              0.030002

auxiliary_verbs
                       scp
highly_central_a          
False            -0.003911
True              0.004550



,scp
admin_a,
False,0.005012
True,0.012089
